# Import packages

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso, LogisticRegression, Ridge, RidgeCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestClassifier, RandomForestRegressor, StackingRegressor, GradientBoostingClassifier
import xgboost as xgb

read two data train and test

In [9]:
train_df = pd.read_csv('/workspaces/House-Prices---Advanced-Regression-Techniques/data/train.csv')
test_df = pd.read_csv('/workspaces/House-Prices---Advanced-Regression-Techniques/data/test.csv')